In [170]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime
from keras.callbacks import History
history = History()
from jinja2 import Environment, FileSystemLoader
import json
import numpy as np
import os
import pandas as pd
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Parameters to specify

In [171]:
#example: Q16.8 -> precision = 8, wordsize = 24, double = 48
#specify:
PRECISION = 16
INTEGER = 16

number_of_input_switches = 1
number_of_hidden_switches = 1
number_of_output_switches = 1

WORDSIZE = INTEGER + PRECISION
D_WORDSIZE = 2*WORDSIZE
SLACK = 8-(WORDSIZE%8)

n_bits = WORDSIZE
wordsize = WORDSIZE
precision = PRECISION
slack = SLACK

nn_topo_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-reports/nn_model_parameters_8x8x4.json"
load_model_title = '/content/drive/MyDrive/network-traffic-classification-main/tf-models/nn_trained_model_weights_8x8x4.csv'
path_test = f'/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_test_2000_samples_4_classes.csv'

### Load NN parameters and model

In [172]:
#Load file with NN topology
with open(nn_topo_title, "r") as f:
  rep = json.load(f)
print(rep)

#Get NN topology and information to load correct dataframes
attributes_array = rep["attributes"]
number_of_attributes = rep["number_of_attributes"]
hidden_layer_nodes = rep["hidden_layer_nodes"]
output_layer_nodes = rep["output_layer_nodes"]
accuracy_test = rep["accuracy_test"]
batch_size = rep["batch_size"]
epochs = rep["epochs"]
number_of_samples = 2000
df_number = 1

#load dataframes and adequate them for file generation
#path_train = '/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_train_with%d'%number_of_samples + 'samples_per_class%d'%df_number + '.csv'
#df_train_raw = pd.read_csv(path_train)
df_test_raw = pd.read_csv(path_test)
#display(df_test_raw)
df_test = df_test_raw[attributes_array]
display(df_test)
target_test = df_test_raw['target']


#load model to get weights
print(load_model_title)
model = tf.keras.models.load_model(load_model_title)

norm_layer_weights = model.layers[0].get_weights()[0] #weight
norm_layer_biases  = model.layers[0].get_weights()[1] #bias
hidden_layer_weights = model.layers[1].get_weights()[0] #weight
hidden_layer_biases  = model.layers[1].get_weights()[1] #bias
out_layer_weights = model.layers[2].get_weights()[0] #weight
out_layer_biases  = model.layers[2].get_weights()[1] #bias
print('\nInput -> Normalization Layer bias:\n',norm_layer_biases)
print('\nInput -> Normalization Layer weights:\n',norm_layer_weights)
print('\nNormalization -> Hidden Layer bias:\n',hidden_layer_biases)
print('\nNormalization -> Hidden Layer weights:\n',hidden_layer_weights)
print('\nHidden -> Output Layer bias:\n',out_layer_biases)
print('\nHidden -> Output Layer weights:\n',out_layer_weights,'\n')

{'datetime': '23-10-2023 17:28:55', 'attributes': ['82', '83', '81', '89', '15', '22', '84', '90'], 'number_of_attributes': 8, 'hidden_layer_nodes': 8, 'output_layer_nodes': 4, 'accuracy_test': 0.8690000176429749, 'attributes_array': ['82', '83', '81', '89', '15', '22', '84', '90', '88', '79', '87', '80', '17', '64', '60', '10', '29', '59', '63', '24', '62', '61', '91', '48', '47', '32', '34', '31', '33', '45', '46', '36', '37', '38', '92'], 'batch_size': 10, 'epochs': 10}


,82,83,81,89,15,22,84,90
0,189,485,485,5840,551,537,189,9651
1,0,267,267,24820,321,307,0,8493
2,248,746,747,49640,801,787,248,32120
3,200,3,1460,65102,1514,1500,21,57344
4,44,3,536,33202,590,576,16,5840
...,...,...,...,...,...,...,...,...
7995,150,190,190,16384,244,230,150,8090
7996,73,6,1380,7955,1434,1420,21,54532
7997,99,22,68,16287,153,139,18,16362
7998,0,0,0,16384,62,48,0,0


/content/drive/MyDrive/network-traffic-classification-main/tf-models/nn_trained_model_weights_8x8x4.csv

Input -> Normalization Layer bias:
 [1.9368844e+05 5.5130387e+04 1.6639320e+05 4.5333952e+08 2.2405806e+05
 2.2413980e+05 6.4357887e+04 6.3315552e+08]

Input -> Normalization Layer weights:
 [  305.93265   208.22992   478.2172  22379.758     682.77155   668.7062
   147.78061 31500.838  ]

Normalization -> Hidden Layer bias:
 [ 0.7125071   0.11499243  0.35747856 -0.24265182 -0.416695   -0.88116324
 -0.07339249  1.9354496 ]

Normalization -> Hidden Layer weights:
 [[ 0.6324756  -2.5168035  -0.21646641 -2.3255978  -0.8245     -1.5634109
  -0.3810429   0.9806638 ]
 [ 1.2927959  -0.425458   -1.1379769   1.6472976   0.33872607  0.83027583
  -2.4607995   1.3243811 ]
 [-0.2056666  -0.30766514 -1.0697153   0.28661203  0.14133376 -0.05261394
   0.7061918   0.78249985]
 [ 0.2017779   0.36118403  0.02930174 -0.00594174 -0.02779741 -0.44857
  -0.16060095 -0.10707805]
 [ 0.07923908  0.2027724   0

In [173]:

print(out_layer_weights[0][0])
print(out_layer_weights[2][0])
print(out_layer_weights[3][0])
print(out_layer_weights[4][0])



0.2875832
-1.3899902
0.1900125
-0.9513979


In [174]:
max_number_of_neurons = max(number_of_attributes,hidden_layer_nodes,output_layer_nodes)

In [175]:
def num_to_c2(num, wordsize):
  if(num < 0):
    return 2**wordsize + num
  return num

In [176]:
# in this 8x8x8 neural network, we will be working with just the 8 classes with most samples (original dataset has 12 classes).
# therefore we already dropped drop classes 4, 8, 10 and 11
# because of this, need to rename the remaining classes so we keep a list from 0 to 7.
# class 9 (SERVICES) must become class 4.

for i,value in enumerate(target_test):
    if value == 5:
      target_test[i] = 2
    if value == 7:
      target_test[i] = 3

### file generator for P4 csv input

In [177]:
df_test.to_csv(f"/content/drive/MyDrive/network-traffic-classification-main/code/csv_files_input/df_test_{df_number}_with_{number_of_attributes}_attributes.csv", index=False)
target_test.to_csv(f"/content/drive/MyDrive/network-traffic-classification-main/csv-files/test_target_{df_number}_with_{number_of_attributes}_attributes.csv", index=False)


###  file generator for TF predictions
to be compared to P4_prediction in the analiser

In [178]:
tf_predictions_probabilities = model.predict(df_test)
tf_loss, tf_acc =  model.evaluate(df_test,  target_test, verbose=2, batch_size=batch_size)

tf_predictions = []

for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)
  j_max = x.argmax()
  tf_predictions.append(j_max)

conf_m = tf.math.confusion_matrix(target_test,tf_predictions)
print(conf_m)

df_tf = pd.DataFrame(tf_predictions_probabilities)
df_tf.insert(loc = 0, column = "tf_predictions", value = tf_predictions)
#print(df_tf)

title_for_analiser = f"/content/drive/MyDrive/network-traffic-classification-main/csv-files/tf_out_class_{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.csv"
df_tf.to_csv(title_for_analiser, index=False)


250/250 [==============================] - 0s 959us/step
800/800 - 1s - loss: 0.3430 - accuracy: 0.8690 - 723ms/epoch - 903us/step
tf.Tensor(
[[1671    8  261   60]
 [   0 1892   87   21]
 [ 356   81 1487   76]
 [   0   41   57 1902]], shape=(4, 4), dtype=int32)


### Topology file generator for sx

In [179]:
# could make all this processing using the same auxiliar value but I choose not, for debug and visualization reasons.
norm_mean = np.zeros(number_of_attributes)
norm_std = np.zeros(number_of_attributes)
norm_mean_fp = np.zeros(number_of_attributes)
norm_std_fp = np.zeros(number_of_attributes)
norm_mean_c2 = np.zeros(number_of_attributes)
norm_std_c2 = np.zeros(number_of_attributes)
weights = norm_layer_weights
biases = norm_layer_biases
act_params = {}

# conversion to fixed point notation Qint.frac
for i in range(number_of_attributes):
  print(f"i:{i},x:{x}")
  norm_mean[i] = -(weights[i])
  norm_std[i] = (math.sqrt(biases[i])) ** -1
  norm_mean_fp[i] = int(norm_mean[i] * (2**PRECISION))
  norm_std_fp[i] = int(norm_std[i] * (2**PRECISION))
  norm_mean_c2[i] = num_to_c2(norm_mean_fp[i], n_bits)
  norm_std_c2[i] = num_to_c2(norm_std_fp[i], n_bits)
  act_params[f"neuron_{i + 1}_mean"] = int(f"{norm_mean_c2[i]:.0f}")
  act_params[f"neuron_{i + 1}_std"] = int(f"{norm_std_c2[i]:.0f}")

#print(f"weight: {weights}, norm_mean:{norm_mean}, norm_mean_fp:{norm_mean_fp}, norm_mean_c2:{norm_mean_c2}")
#print(f"bias: {biases}, norm_std:{norm_std}, norm_std_fp:{norm_std_fp}, norm_std_c2:{norm_std_c2}")

#number of norm_layer_nodes = number_of_attributes
for i in range(number_of_input_switches):
  sx_id = 1+i

  sx_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": 1
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": sx_id
        }
      },
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": sx_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": 1
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": 1
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 5
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 2
        }
      },
      {
        "table": "MyIngress.tab_norm_mean_std",
        "default_action": True,
        "action_name": "MyIngress.set_norm_mean_std",
        "action_params": act_params
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : sx_id,
        "replicas": []
      }
    ]
  }
  for h in range(number_of_hidden_switches):
    sx_entries["multicast_group_entries"][0]["replicas"].append(
      {
        "egress_port" :51+h,
        "instance": 1
      }
    )


  print(sx_entries["table_entries"])
  sx_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%sx_id + "-runtime.json"
  with open(sx_file_title, "w") as f:
    json.dump(sx_entries, f, indent = 2)

i:0,x:[-4.25767   -1.1758502  1.657073  -1.3072054]
i:1,x:[-4.25767   -1.1758502  1.657073  -1.3072054]
i:2,x:[-4.25767   -1.1758502  1.657073  -1.3072054]
i:3,x:[-4.25767   -1.1758502  1.657073  -1.3072054]
i:4,x:[-4.25767   -1.1758502  1.657073  -1.3072054]
i:5,x:[-4.25767   -1.1758502  1.657073  -1.3072054]
i:6,x:[-4.25767   -1.1758502  1.657073  -1.3072054]
i:7,x:[-4.25767   -1.1758502  1.657073  -1.3072054]
[{'table': 'MyIngress.ann_forward', 'match': {'standard_metadata.ingress_port': 1}, 'action_name': 'MyIngress.mcast', 'action_params': {'mgroup': 1}}, {'table': 'MyIngress.tab_neuron_id', 'default_action': True, 'action_name': 'MyIngress.set_neuron_id', 'action_params': {'neuron_id': 1}}, {'table': 'MyIngress.tab_n_expected_stimuli', 'default_action': True, 'action_name': 'MyIngress.set_n_expected_stimuli', 'action_params': {'n_expected_stimuli': 1}}, {'table': 'MyIngress.tab_expected_stimuli', 'default_action': True, 'action_name': 'MyIngress.set_expected_stimuli', 'action_par

### Topology file generator for s5x

In [180]:
# Hidden Layer Switches

# Calculate bias values using Fixed Point (Qinteger.precision) and Two's Complement (C2) and for
bias_params = {}
biases = hidden_layer_biases
fp_bias_value = np.zeros(hidden_layer_nodes)
c2_bias_value = np.zeros(hidden_layer_nodes)
for i in range(hidden_layer_nodes):
  fp_bias_value[i] = int(biases[i] * (2**PRECISION))
  c2_bias_value[i] = num_to_c2(fp_bias_value[i], n_bits)
  bias_params[f"neuron_{i + 1}_bias"] = int(f"{c2_bias_value[i]:.0f}")
# print(c2_bias_value)

# Calculate weights
weights_params = {}
weights = hidden_layer_weights
fp_value = np.zeros((number_of_attributes,hidden_layer_nodes))
c2_value = np.zeros((number_of_attributes,hidden_layer_nodes))
for i in range(hidden_layer_nodes):
  for j in range(number_of_attributes):
    fp_value[j][i] = int(weights[j][i] * (2**PRECISION))
    c2_value[j][i] = num_to_c2(fp_value[j][i], n_bits)
    weights_params[f"n2n_{i + 1}_weight_{j + 1}"] = int(f"{c2_value[j][i]:.0f}")
#print(c2_value)

# Calculate expected stimuli
expected_stimuli = sum([1<<(i + 1) for i in range(number_of_attributes)])

# Pass information to switch tables
for i in range(number_of_hidden_switches):
  s5x_id = 51+i
  s5x_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": s5x_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": number_of_input_switches
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": expected_stimuli
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 1
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 3
        }
      },
      {
        "table": "MyIngress.tab_neuron_bias",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_bias",
        "action_params": bias_params
      },
      {
        "table": "MyIngress.tab_n2n_weight",
        "match": {
          "hdr.ann.neuron_id": j+1
        },
        "action_name": "MyIngress.set_n2n_weight",
        "action_params": weights_params
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : s5x_id,
        "replicas":[]
      }
    ]
  }
  for j in range(number_of_input_switches):
    s5x_entries["table_entries"].append(
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": j+1
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": s5x_id
        }
      }
    )
  for h in range(number_of_output_switches):
    s5x_entries["multicast_group_entries"][0]["replicas"].append(
      {
        "egress_port" :101+h,
        "instance": 1
      }
    )

  s5x_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%s5x_id + "-runtime.json"
  with open(s5x_file_title, "w") as f:
    json.dump(s5x_entries, f, indent = 2)

print(json.dumps(s5x_entries, indent = 2))

{
  "target": "bmv2",
  "p4info": "build/ANN.p4.p4info.txt",
  "bmv2_json": "build/ANN.json",
  "table_entries": [
    {
      "table": "MyIngress.tab_neuron_id",
      "default_action": true,
      "action_name": "MyIngress.set_neuron_id",
      "action_params": {
        "neuron_id": 51
      }
    },
    {
      "table": "MyIngress.tab_n_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_n_expected_stimuli",
      "action_params": {
        "n_expected_stimuli": 1
      }
    },
    {
      "table": "MyIngress.tab_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_expected_stimuli",
      "action_params": {
        "expected_stimuli": 510
      }
    },
    {
      "table": "MyIngress.tab_agg_func",
      "default_action": true,
      "action_name": "MyIngress.set_agg_func",
      "action_params": {
        "agg_func": 1
      }
    },
    {
      "table": "MyIngress.tab_activation_func",
      "default_action": tru

### Topology file generator for s1xx

In [201]:
# Output Layer Switches

# Calculate bias values using Fixed Point (Qinteger.precision) and Two's Complement (C2)
bias_params = {}
biases = out_layer_biases
fp_bias_value = np.zeros(output_layer_nodes)
c2_bias_value = np.zeros(output_layer_nodes)
for i in range(output_layer_nodes):
  fp_bias_value[i] = int(biases[i] * (2**PRECISION))
  c2_bias_value[i] = num_to_c2(fp_bias_value[i], n_bits)
  bias_params[f"neuron_{i + 1}_bias"] = int(f"{c2_bias_value[i]:.0f}")

for i in range(max_number_of_neurons - output_layer_nodes):
  bias_params[f"neuron_{output_layer_nodes + 1}_bias"] = 0
#print(c2_bias_value)

weights_params = {}
weights = out_layer_weights
fp_value = np.zeros((hidden_layer_nodes,output_layer_nodes))
c2_value = np.zeros((hidden_layer_nodes,output_layer_nodes))
for i in range(output_layer_nodes):
  for j in range(hidden_layer_nodes):
    fp_value[j][i] = int(weights[j][i] * (2**PRECISION))
    c2_value[j][i] = num_to_c2(fp_value[j][i], n_bits)
    weights_params[f"n2n_{i + 1}_weight_{j + 1}"] = int(f"{c2_value[j][i]:.0f}")
#print(c2_value)

# Calculate expected stimuli
expected_stimuli = sum([1<<(i + 51) for i in range(hidden_layer_nodes)])

# Pass information to switch tables
for i in range(number_of_output_switches):
  s1xx_id = 101+i
  s1xx_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": s1xx_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": number_of_hidden_switches
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": expected_stimuli

        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 6
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 2
        }
      },
      {
        "table": "MyIngress.tab_neuron_bias",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_bias",
        "action_params": bias_params
      },
      {
        "table": "MyIngress.tab_n2n_weight",
        "match": {
          "hdr.ann.neuron_id": j+1
        },
        "action_name": "MyIngress.set_n2n_weight",
        "action_params": weights_params
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : s1xx_id,
        "replicas":[
          {
            "egress_port" :126,
            "instance": 1
          }
        ]
      }
    ]
  }
  for j in range(number_of_input_switches):
    s1xx_entries["table_entries"].append(
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": j+51
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": s1xx_id
        }
      }
    )
  #print(s1xx_entries["table_entries"])
  s1xx_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%s1xx_id + "-runtime.json"
  with open(s1xx_file_title, "w") as f:
    json.dump(s1xx_entries, f, indent = 2)

print(json.dumps(s1xx_entries, indent = 2))

{
  "target": "bmv2",
  "p4info": "build/ANN.p4.p4info.txt",
  "bmv2_json": "build/ANN.json",
  "table_entries": [
    {
      "table": "MyIngress.tab_neuron_id",
      "default_action": true,
      "action_name": "MyIngress.set_neuron_id",
      "action_params": {
        "neuron_id": 101
      }
    },
    {
      "table": "MyIngress.tab_n_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_n_expected_stimuli",
      "action_params": {
        "n_expected_stimuli": 1
      }
    },
    {
      "table": "MyIngress.tab_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_expected_stimuli",
      "action_params": {
        "expected_stimuli": 574208952489738240
      }
    },
    {
      "table": "MyIngress.tab_agg_func",
      "default_action": true,
      "action_name": "MyIngress.set_agg_func",
      "action_params": {
        "agg_func": 6
      }
    },
    {
      "table": "MyIngress.tab_activation_func",
      "defa

### P4 File Generation: defines, headers, parser, checksum

In [182]:
headers = """
/* -*- P4_16 -*- */
#include <core.p4>
#include <v1model.p4>

const bit<16> TYPE_IPV4 = 0x800;

#define ET_ANN 0x88B5

#define FUNC_WEIGHTED_SUM_8_TO_8 1
#define FUNC_IDENTITY 2
#define FUNC_RELU 3
#define FUNC_ARGMAX 4
#define FUNC_NORMALIZATION 5
#define FUNC_WEIGHTED_SUM_8_TO_4 6

#define PRECISION 16
#define WORDSIZE 32
#define D_WORDSIZE 64
#define SLACK 8

/*************************************************************************
*********************** H E A D E R S  ***********************************
*************************************************************************/

typedef bit<9>  egressSpec_t;
typedef bit<48> macAddr_t;
typedef bit<32> ip4Addr_t;

header ethernet_t{
    macAddr_t dstAddr;
    macAddr_t srcAddr;
    bit<16>   etherType;
}

header ann_t{
    bit<32> neuron_id;
"""
for i in range(max_number_of_neurons):
  headers += f"    bit<WORDSIZE> data_{i+1};\n"
headers += """    bit<16> run_id;
    bit<SLACK> slack;
}

struct metadata{
    bit<32> neuron_id;            // temporarily stores the ID of the neuron in this switch.
    bit<32> n_expected_stimuli;   // temporarily stores the number of expected stimuli by the neuron in a single ANN run.
    bit<32> n_received_stimuli;   // temporarily stores the number of stimuli already received by the neuron in the current ANN run.
    bit<128> expected_stimuli;    // temporarily stores a bitstring that indicates from which neurons is the neuron expected to receive stimuli. For example, if the bitstring has value 0b1010, the neuron is expected to receive stimuli from neurons with ID 1 and 3, but not from IDs 0 and 2.
    bit<128> received_stimuli;    // temporarily stores a bitstring that indicates from which neurons is the neuron already received stimuli in the current ANN run.

    bit<32> agg_func;
    bit<32> activation_func;
    bit<16> run_id;

    // Stores the data to be processed and fowarded
"""
for i in range(max_number_of_neurons):
  headers += f"    bit<WORDSIZE> neuron_{i+1}_data;\n"

headers += f"\n    // Agg func = normalization\n"
for i in range(number_of_attributes):
  headers += f"    bit<WORDSIZE> neuron_{i+1}_mean;\n"
  headers += f"    bit<WORDSIZE> neuron_{i+1}_std;\n"

headers += f"\n    // Agg func = weighted sum\n"
for i in range(hidden_layer_nodes):
  headers += f"    bit<WORDSIZE> neuron_{i+1}_bias;\n"

for i in range(hidden_layer_nodes):
  for j in range(number_of_attributes):
    headers += f"    bit<WORDSIZE> n2n_{i+1}_weight_{j+1};\n"

headers += """
// Agg func = argmax
    bit<WORDSIZE> neuron_max_value;
}

struct headers{
    ethernet_t   ethernet;
    ann_t   ann;
}

/*************************************************************************
*********************** P A R S E R  ***********************************
*************************************************************************/

parser MyParser(packet_in packet,
            out headers hdr,
            inout metadata meta,
            inout standard_metadata_t standard_metadata){

    state start{
        transition parse_ethernet;
    }

    state parse_ethernet{
        packet.extract(hdr.ethernet);
        transition select(hdr.ethernet.etherType){
           ET_ANN: parse_ann;
           default: accept;
        }
    }

    state parse_ann{
      packet.extract(hdr.ann);
      transition accept;
    }
}

/*************************************************************************
************   C H E C K S U M    V E R I F I C A T I O N   *************
*************************************************************************/

control MyVerifyChecksum(inout headers hdr, inout metadata meta){
    apply {  }
}
"""
print(headers)


/* -*- P4_16 -*- */
#include <core.p4>
#include <v1model.p4>

const bit<16> TYPE_IPV4 = 0x800;

#define ET_ANN 0x88B5

#define FUNC_WEIGHTED_SUM_8_TO_8 1
#define FUNC_IDENTITY 2
#define FUNC_RELU 3
#define FUNC_ARGMAX 4
#define FUNC_NORMALIZATION 5
#define FUNC_WEIGHTED_SUM_8_TO_4 6

#define PRECISION 16
#define WORDSIZE 32
#define D_WORDSIZE 64
#define SLACK 8

/*************************************************************************
*********************** H E A D E R S  ***********************************
*************************************************************************/

typedef bit<9>  egressSpec_t;
typedef bit<48> macAddr_t;
typedef bit<32> ip4Addr_t;

header ethernet_t{
    macAddr_t dstAddr;
    macAddr_t srcAddr;
    bit<16>   etherType;
}

header ann_t{
    bit<32> neuron_id;
    bit<WORDSIZE> data_1;
    bit<WORDSIZE> data_2;
    bit<WORDSIZE> data_3;
    bit<WORDSIZE> data_4;
    bit<WORDSIZE> data_5;
    bit<WORDSIZE> data_6;
    bit<WORDSIZE> data_7;
    bit<WOR

### Ingress, tables and actions

In [183]:
ingress = """
/*************************************************************************
**************  I N G R E S S   P R O C E S S I N G   *******************
*************************************************************************/

control MyIngress(inout headers hdr,
                inout metadata meta,
                inout standard_metadata_t standard_metadata) {

    register<bit<32>>(1) reg_n_received_stimuli;
    register<bit<128>>(1) reg_received_stimuli;
"""
for i in range(max_number_of_neurons):
  ingress += f"    register<bit<WORDSIZE>>(1) reg_neuron_{i+1}_data;\n"
ingress +="""
    register<bit<WORDSIZE>>(1) reg_neuron_max_value;
    register<bit<16>>(1) reg_run_id;
    action drop(){
    mark_to_drop(standard_metadata);
}

action mcast(bit<16> mgroup){
    standard_metadata.mcast_grp = mgroup;
}

table ann_forward{
    key = {
        standard_metadata.ingress_port: exact;
    }
    actions = {
        mcast;
        drop;
    }
    size = 1024;
    default_action = drop();
}

action set_neuron_id(bit<32> neuron_id){
    meta.neuron_id = neuron_id;
}

table tab_neuron_id{
    actions = {
        set_neuron_id;
    }
    size = 1;
}

action set_n_expected_stimuli(bit<32> n_expected_stimuli){
    meta.n_expected_stimuli = n_expected_stimuli;
}

table tab_n_expected_stimuli{
    actions = {
        set_n_expected_stimuli;
    }
    size = 1;
}

action set_expected_stimuli(bit<128> expected_stimuli){
    meta.expected_stimuli = expected_stimuli;
}

table tab_expected_stimuli{
    actions = {
        set_expected_stimuli;
    }
    size = 1;
}

action set_agg_func(bit<32> agg_func){
    meta.agg_func = agg_func;
}

table tab_agg_func{
    actions = {
        set_agg_func;
    }
    size = 1;
}

action set_neuron_bias("""
for i in range(hidden_layer_nodes):
  ingress += f"bit<WORDSIZE> neuron_{i+1}_bias, "
ingress = ingress[:-2]
ingress += "){\n"

for i in range(hidden_layer_nodes):
  ingress += f"   meta.neuron_{i+1}_bias = neuron_{i+1}_bias;\n"
ingress += """}

table tab_neuron_bias{
    actions = {
        set_neuron_bias;
    }
    size = 1;
}

action set_n2n_weight("""
for i in range(hidden_layer_nodes):
  for j in range(number_of_attributes):
    ingress += f"bit<WORDSIZE> n2n_{i+1}_weight_{j+1}, "
ingress = ingress[:-2]

ingress += "){\n"
for i in range(hidden_layer_nodes):
  for j in range(number_of_attributes):
    ingress += f"   meta.n2n_{i+1}_weight_{j+1} = n2n_{i+1}_weight_{j+1};\n"

ingress += """}

table tab_n2n_weight{
    key = {
        hdr.ann.neuron_id: exact;
    }
    actions = {
        set_n2n_weight;
    }
    size = 256;
}

action set_norm_mean_std("""
for i in range(number_of_attributes):
  ingress += f"bit<WORDSIZE> neuron_{i+1}_mean, bit<WORDSIZE> neuron_{i+1}_std, "
ingress = ingress[:-2]
ingress += "){\n"
for i in range(number_of_attributes):
  ingress += f"   meta.neuron_{i+1}_mean = neuron_{i+1}_mean;\n"
  ingress += f"   meta.neuron_{i+1}_std = neuron_{i+1}_std;\n"
ingress += """
}

table tab_norm_mean_std{
    actions = {
        set_norm_mean_std;
    }
    size = 1;
}

action set_activation_func(bit<32> activation_func){
    meta.activation_func = activation_func;
}

table tab_activation_func{
    actions = {
        set_activation_func;
    }
    size = 1;
}
"""
print(ingress)


/*************************************************************************
**************  I N G R E S S   P R O C E S S I N G   *******************
*************************************************************************/

control MyIngress(inout headers hdr,
                inout metadata meta,
                inout standard_metadata_t standard_metadata) {

    register<bit<32>>(1) reg_n_received_stimuli;
    register<bit<128>>(1) reg_received_stimuli;
    register<bit<WORDSIZE>>(1) reg_neuron_1_data;
    register<bit<WORDSIZE>>(1) reg_neuron_2_data;
    register<bit<WORDSIZE>>(1) reg_neuron_3_data;
    register<bit<WORDSIZE>>(1) reg_neuron_4_data;
    register<bit<WORDSIZE>>(1) reg_neuron_5_data;
    register<bit<WORDSIZE>>(1) reg_neuron_6_data;
    register<bit<WORDSIZE>>(1) reg_neuron_7_data;
    register<bit<WORDSIZE>>(1) reg_neuron_8_data;

    register<bit<WORDSIZE>>(1) reg_neuron_max_value;
    register<bit<16>>(1) reg_run_id;
    action drop(){
    mark_to_drop(standard_met

### Apply

In [184]:
apply = """
apply {
    if(hdr.ann.isValid()){                                   // If the ANN header is present in the packet
        reg_run_id.read(meta.run_id, 0);
        if(hdr.ann.run_id != meta.run_id){                   // If the run_id in the receiving differs from the stored run_id, reset the received stimuli so we don't mix up data
            reg_run_id.write(0, hdr.ann.run_id);
            reg_received_stimuli.write(0, 0);
            reg_n_received_stimuli.write(0, 0);
        }
        tab_expected_stimuli.apply();                         // Get the bitstring of expected stimuli and store in the MD field
        reg_received_stimuli.read(meta.received_stimuli, 0);  // Get the bitstring of received stimuli and store in the MD field

        // Declare and compute the value of a variable that indicates whether the stimulus in the packet is expected
        bit<128> expected = meta.expected_stimuli & ((bit<128>) 1 << (bit<8>) hdr.ann.neuron_id); // the bit shift and & operator enable us to do the checking.
        // Declare and compute the value of a variable that indicates whether the stimulus in the packet has been received
        bit<128> received = meta.received_stimuli & ((bit<128>) 1 << (bit<8>) hdr.ann.neuron_id);

        // Check if the stimulus is expected and was not yet received
        if((expected > (bit<128>) 0) && (received == (bit<128>) 0)){
            meta.received_stimuli = meta.received_stimuli | ((bit<128>) 1 << (bit<8>) hdr.ann.neuron_id);
            reg_received_stimuli.write(0, meta.received_stimuli);
            // Load n_received_stimuli from register, increment it, and write back
            reg_n_received_stimuli.read(meta.n_received_stimuli, 0);
            meta.n_received_stimuli = meta.n_received_stimuli + 1;
            reg_n_received_stimuli.write(0, meta.n_received_stimuli);
            // Set the register(s) storing the neuron aggregation and bias function
            tab_agg_func.apply();
            tab_neuron_bias.apply();
"""

### Normalization

In [185]:
normalization = """
            //Calculate the aggregation funciton

            if(meta.agg_func == FUNC_NORMALIZATION){
                // normalized_value = (raw_value - weight) / sqrt(biases)
                // since there's no subtraction nor division in P4, must adequate the formula to
                // normalized_value = (raw_value + (-weight) * (sqrt(bias)) ** -1

                // A NOTE ON SIGN EXTENSION: When we extend the number of bits of a negative number, we must also extend the signal to keep the correctness.
                // Example	-Corect: positive, no need for sign extension	w: 0001 -> dw: 0000 0001
                //			-Corect: negative, with sign extension 			w: 1110 -> dw: 1111 1110
                //			-WRONG:  negative, without sign extension 		w: 1110 -> dw: 0000 1110

                tab_norm_mean_std.apply(); // Load weight (mean) and bias (std)
"""
for i in range(number_of_attributes):
  normalization += f"""
                // Neuron {i+1}:
                bit<WORDSIZE> operand_a{i+1} = hdr.ann.data_{i+1} << PRECISION; // Pass the values to registers to be able to operate them. To load the input data, which are integers, need to shift left to adequate them to FP notation Q.INT.FRAC. TO_DO need special treatment to NEGATIVE INPUT DATA!!!
                bit<WORDSIZE> operand_b{i+1} = meta.neuron_{i+1}_mean; // Load normalization means (= weights)
                bit<WORDSIZE> sum_result_{i+1} = operand_a{i+1} + operand_b{i+1}; // Compute the sum
                bit<D_WORDSIZE> sum_result_{i+1}_dw = (bit<D_WORDSIZE>) sum_result_{i+1}; // Need to double the wordsize to store the multiplication result
                if((sum_result_{i+1}_dw & (1 << (WORDSIZE-1))) > 0){{                            // negative number
                    sum_result_{i+1}_dw = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) sum_result_{i+1}_dw;
                }}
                bit<D_WORDSIZE> operand_c{i+1} = (bit<D_WORDSIZE>) meta.neuron_{i+1}_std;  // Load normalization std (= bias)
                // Sign extension
                if((operand_c{i+1} & (1 << (WORDSIZE-1))) > 0){{                                // negative number
                    operand_c{i+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_c{i+1};
                }}
                bit<D_WORDSIZE> norm_result_{i+1} = ((sum_result_{i+1}_dw * operand_c{i+1}) >> PRECISION);
                meta.neuron_{i+1}_data = (bit<WORDSIZE>) norm_result_{i+1}; // Resize the data to wordsize to be fowarded
                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);  // Store the value to be fowarded
"""
normalization += "            }"
print(normalization)


            //Calculate the aggregation funciton

            if(meta.agg_func == FUNC_NORMALIZATION){
                // normalized_value = (raw_value - weight) / sqrt(biases)
                // since there's no subtraction nor division in P4, must adequate the formula to
                // normalized_value = (raw_value + (-weight) * (sqrt(bias)) ** -1

                // A NOTE ON SIGN EXTENSION: When we extend the number of bits of a negative number, we must also extend the signal to keep the correctness.
                // Example	-Corect: positive, no need for sign extension	w: 0001 -> dw: 0000 0001
                //			-Corect: negative, with sign extension 			w: 1110 -> dw: 1111 1110
                //			-WRONG:  negative, without sign extension 		w: 1110 -> dw: 0000 1110

                tab_norm_mean_std.apply(); // Load weight (mean) and bias (std)

                // Neuron 1:
                bit<WORDSIZE> operand_a1 = hdr.ann.data_1 << PRECISION; // Pass the values to regi

### Weighted Sum

In [186]:
def w_sum_gen(previous_layer_nodes, this_layer_nodes):
  w_sum = f"""
            else if(meta.agg_func == FUNC_WEIGHTED_SUM_{previous_layer_nodes}_TO_{this_layer_nodes}){{                // Aggregation Function = weighted sum = bias + Summation_i=1_to_n(data_i * weight_i)
                if(meta.n_received_stimuli == 1){{ // Check if this is the first stimulus in an ANN run
                    // If yes, initialize neuron_data with the neuron bias, the neuron bias is added to the accumulator (neuron_data) just once
"""
  for i in range(this_layer_nodes):
    w_sum += f"                    meta.neuron_{i+1}_data = meta.neuron_{i+1}_bias;\n"
  w_sum += """                }
                else{
                    // If not, read the neuron_data value stored in the register
"""
  for i in range(this_layer_nodes):
    w_sum += f"                    reg_neuron_{i+1}_data.read(meta.neuron_{i+1}_data, 0);\n"
  w_sum += """                }
                tab_n2n_weight.apply();	// Get the neuron weights
                // Load data and perform sign extension
"""
  for i in range(previous_layer_nodes):
    w_sum += f"""
                bit<D_WORDSIZE> operand_b{i+1} = (bit<D_WORDSIZE>) hdr.ann.data_{i+1};
                // Sign extension
                if((operand_b{i+1} & (1 << (WORDSIZE-1))) > 0){{ // negative number
                    operand_b{i+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_b{i+1};
                }}"""

  for i in range(this_layer_nodes):
    w_sum += f"""
                // Neuron {i+1}:
                // Pass values to aux variable to be able to operate them
"""
    for j in range(previous_layer_nodes):
      w_sum += f"                bit<D_WORDSIZE> operand_a_{i+1}_{j+1} = (bit<D_WORDSIZE>) meta.n2n_{i+1}_weight_{j+1};\n"
    w_sum += f"                // Sign extension"
    for j in range(previous_layer_nodes):
      w_sum += f"""
                if((operand_a_{i+1}_{j+1} & (1 << (WORDSIZE-1))) > 0){{ // negative number
                    operand_a_{i+1}_{j+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_a_{i+1}_{j+1};
                }}"""

    w_sum += f"\n                // perform multiplication and shift to transform data back to wordsize\n"
    for j in range(previous_layer_nodes):
      w_sum += f"                bit<D_WORDSIZE> res_{i+1}_{j+1} = ((operand_a_{i+1}_{j+1} * operand_b{j+1}) >> PRECISION);\n"
    w_sum += f"""
                // Compute the summation
                meta.neuron_{i+1}_data = meta.neuron_{i+1}_data"""
    for j in range(previous_layer_nodes):
      w_sum += f" + (bit<WORDSIZE>) res_{i+1}_{j+1}"
    w_sum += f""";
                // Store data to be fowarded
                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);
"""
  w_sum += "            }"
  return w_sum

w_sum_str = w_sum_gen(number_of_attributes, hidden_layer_nodes)
w_sum_str += w_sum_gen(hidden_layer_nodes, output_layer_nodes)

print(w_sum_str)


            else if(meta.agg_func == FUNC_WEIGHTED_SUM_8_TO_8){                // Aggregation Function = weighted sum = bias + Summation_i=1_to_n(data_i * weight_i)
                if(meta.n_received_stimuli == 1){ // Check if this is the first stimulus in an ANN run
                    // If yes, initialize neuron_data with the neuron bias, the neuron bias is added to the accumulator (neuron_data) just once
                    meta.neuron_1_data = meta.neuron_1_bias;
                    meta.neuron_2_data = meta.neuron_2_bias;
                    meta.neuron_3_data = meta.neuron_3_bias;
                    meta.neuron_4_data = meta.neuron_4_bias;
                    meta.neuron_5_data = meta.neuron_5_bias;
                    meta.neuron_6_data = meta.neuron_6_bias;
                    meta.neuron_7_data = meta.neuron_7_bias;
                    meta.neuron_8_data = meta.neuron_8_bias;
                }
                else{
                    // If not, read the neuron_data value s

### Argmax

In [187]:
argmax = """
            else if(meta.agg_func == FUNC_ARGMAX){
                // the data to be fowarded (neuron_1_data) is the ID of the switch with highest value.
                // neuron_2_data is the index of the neuron with highest value inside the same switch.
                // the highest data (neuron_max_value) is kept to be compared by other neurons.
                bit<WORDSIZE> op_a = 0;
                bit<WORDSIZE> op_b = 0;
                bit<1> op_a_sig = 0;
                bit<1> op_b_sig = 0;
                if(meta.n_received_stimuli == 1){
                    // if first stimuli, then assume first data received is the higher, then check the remmaining data against it
                    // Neuron 1
                    meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                    meta.neuron_2_data = 0; // neuron_2_data stores the index of the neuron with highest value within same switch
                    meta.neuron_max_value = hdr.ann.data_1;
"""
for i in range(output_layer_nodes-1):
  argmax += f"""
                    // Neuron {i+2}
                    // Check if data is higher than stored max data
                    op_a = hdr.ann.data_{i+2}; 			// op_a is the data being evaluated if it's higher then the stored one (op_b)
                    op_b = meta.neuron_max_value;		// op_b is the store of max value until now
                    op_a_sig = (bit<1>)(op_a & (1 << (WORDSIZE-1)) > 0);
                    op_b_sig = (bit<1>)(op_b & (1 << (WORDSIZE-1)) > 0);
                    // There are two situation in which op_a is bigger then op_b
                    if((op_a_sig == 0) && (op_b_sig  == 1)){{ // The first: if the op_a is positive and op_b is negative
                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                        meta.neuron_2_data = {i+1};
                        meta.neuron_max_value = hdr.ann.data_{i+2};
                    }} else if(op_a_sig == op_b_sig && op_a > op_b){{ // The second: if the signal is the same, and op_a > op_b
                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                        meta.neuron_2_data = {i+1};
                        meta.neuron_max_value = hdr.ann.data_{i+2};
                    }}"""
argmax += """
                }
                else{
                    // this is wrong!?!? reg_neuron_1_data.read(meta.neuron_1_data, 0);
                    reg_neuron_max_value.read(meta.neuron_max_value, 0);
"""
for i in range(output_layer_nodes):
  argmax += f"""
                    // Neuron {i+1}
                    // Check if data is higher than stored max data
                    op_a = hdr.ann.data_{i+1}; 			// op_a is the data being evaluated if it's higher then the stored one (op_b)
                    op_b = meta.neuron_max_value;		// op_b is the store of max value until now
                    op_a_sig = (bit<1>)(op_a & (1 << (WORDSIZE-1)) > 0);
                    op_b_sig = (bit<1>)(op_b & (1 << (WORDSIZE-1)) > 0);
                    // There are two situation in which op_a is bigger then op_b
                    if((op_a_sig == 0) && (op_b_sig  == 1)){{ // The first: if the op_a is positive and op_b is negative
                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                        meta.neuron_2_data = {i};
                        meta.neuron_max_value = hdr.ann.data_{i+1};
                    }} else if(op_a_sig == op_b_sig && op_a > op_b){{ // The second: if the signal is the same, and op_a > op_b
                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                        meta.neuron_2_data = {i};
                        meta.neuron_max_value = hdr.ann.data_{i+1};
                    }}"""

argmax +="""
                }
                reg_neuron_1_data.write(0, meta.neuron_1_data);
                reg_neuron_2_data.write(0, meta.neuron_2_data);
                reg_neuron_max_value.write(0, meta.neuron_max_value);
                // in the argmax function, neuron 3 data and neuron 4 data are not needed, set to 99 just for testing. Could use a different header for this layer so we don't have empty fileds.
"""
for i in range(max_number_of_neurons-2):
  argmax += f"""                meta.neuron_{i+3}_data = 99;
                reg_neuron_{i+3}_data.write(0, meta.neuron_{i+3}_data);
"""
argmax +="            }\n"
print(argmax)



            else if(meta.agg_func == FUNC_ARGMAX){
                // the data to be fowarded (neuron_1_data) is the ID of the switch with highest value.
                // neuron_2_data is the index of the neuron with highest value inside the same switch.
                // the highest data (neuron_max_value) is kept to be compared by other neurons.
                bit<WORDSIZE> op_a = 0;
                bit<WORDSIZE> op_b = 0;
                bit<1> op_a_sig = 0;
                bit<1> op_b_sig = 0;
                if(meta.n_received_stimuli == 1){
                    // if first stimuli, then assume first data received is the higher, then check the remmaining data against it
                    // Neuron 1
                    meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                    meta.neuron_2_data = 0; // neuron_2_data stores the index of the neuron with highest value within same switch
                    meta.neuron_max_value = hdr.ann.data_1;

             

### Identity

In [188]:
ident = "            else if(meta.agg_func == FUNC_IDENTITY){"
for i in range(max_number_of_neurons):
  ident += f"""
                meta.neuron_{i+1}_data = hdr.ann.data_{i+1};
                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);\n"""

ident += "            }"
print(ident)

            else if(meta.agg_func == FUNC_IDENTITY){
                meta.neuron_1_data = hdr.ann.data_1;
                reg_neuron_1_data.write(0, meta.neuron_1_data);

                meta.neuron_2_data = hdr.ann.data_2;
                reg_neuron_2_data.write(0, meta.neuron_2_data);

                meta.neuron_3_data = hdr.ann.data_3;
                reg_neuron_3_data.write(0, meta.neuron_3_data);

                meta.neuron_4_data = hdr.ann.data_4;
                reg_neuron_4_data.write(0, meta.neuron_4_data);

                meta.neuron_5_data = hdr.ann.data_5;
                reg_neuron_5_data.write(0, meta.neuron_5_data);

                meta.neuron_6_data = hdr.ann.data_6;
                reg_neuron_6_data.write(0, meta.neuron_6_data);

                meta.neuron_7_data = hdr.ann.data_7;
                reg_neuron_7_data.write(0, meta.neuron_7_data);

                meta.neuron_8_data = hdr.ann.data_8;
                reg_neuron_8_data.write(0, meta.neuron_8_data);
    

### Activation

In [189]:
activation = """

            //  After computing aggregation functions, check if all stimuli have been received to proced to activation function
            tab_n_expected_stimuli.apply();                             // Get the number of expected stimuli for the neuron
            if(meta.n_received_stimuli == meta.n_expected_stimuli){     // Check if the number of expected stimuli was just reached, if yes, the neuron_data is the final value, we should propagate it
                tab_neuron_id.apply();                                  // Get the neuron ID
                if(meta.neuron_id > 0){
                    hdr.ann.neuron_id = meta.neuron_id;                 // Overwrite the fields in the ANN header
                }
                tab_activation_func.apply();                            // Get the neuron activation function
                if(meta.activation_func == FUNC_RELU){"""
for i in range(max_number_of_neurons):
  activation += f"""
                    if(meta.neuron_{i+1}_data & (1 << (WORDSIZE-1)) > 0){{     // Relu: if negative, set data to 0
                        meta.neuron_{i+1}_data = 0;
                    }}
                    hdr.ann.data_{i+1} = meta.neuron_{i+1}_data;                    // Overwrite the fields in the ANN header
"""
activation += """                }
                else if(meta.activation_func == FUNC_IDENTITY){
"""
for i in range(max_number_of_neurons):
  activation += f"                    hdr.ann.data_{i+1} = meta.neuron_{i+1}_data;                    // Overwrite the fields in the ANN header\n"
activation += """                }

                reg_received_stimuli.write(0, 0);                     // Reset the registers related to received stimuli
                reg_n_received_stimuli.write(0, 0);
                ann_forward.apply();                                    // Forward the packet according to the ANN forwarding logic
            }
            else {
                drop();
            }
        }
    }
}
}

/*************************************************************************
****************  E G R E S S   P R O C E S S I N G   *******************
*************************************************************************/

control MyEgress(inout headers hdr,
                inout metadata meta,
                inout standard_metadata_t standard_metadata) {
    apply {}
}

/*************************************************************************
*************   C H E C K S U M    C O M P U T A T I O N   **************
*************************************************************************/

control MyComputeChecksum(inout headers  hdr, inout metadata meta) {
    apply {}
}

/*************************************************************************
***********************  D E P A R S E R  *******************************
*************************************************************************/

control MyDeparser(packet_out packet, in headers hdr) {
    apply {
        packet.emit(hdr.ethernet);
        packet.emit(hdr.ann);
    }
}

/*************************************************************************
***********************  S W I T C H  *******************************
*************************************************************************/

V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
"""
print(activation)




            //  After computing aggregation functions, check if all stimuli have been received to proced to activation function
            tab_n_expected_stimuli.apply();                             // Get the number of expected stimuli for the neuron
            if(meta.n_received_stimuli == meta.n_expected_stimuli){     // Check if the number of expected stimuli was just reached, if yes, the neuron_data is the final value, we should propagate it
                tab_neuron_id.apply();                                  // Get the neuron ID
                if(meta.neuron_id > 0){
                    hdr.ann.neuron_id = meta.neuron_id;                 // Overwrite the fields in the ANN header
                }
                tab_activation_func.apply();                            // Get the neuron activation function
                if(meta.activation_func == FUNC_RELU){
                    if(meta.neuron_1_data & (1 << (WORDSIZE-1)) > 0){     // Relu: if negative, set data to 0
     

### Full P4 Code


In [190]:
w_sum_str = w_sum_gen(number_of_attributes, hidden_layer_nodes)
w_sum_str += w_sum_gen(hidden_layer_nodes, output_layer_nodes)
code = headers + ingress + apply + normalization + w_sum_str + argmax + ident + activation
print(code)


/* -*- P4_16 -*- */
#include <core.p4>
#include <v1model.p4>

const bit<16> TYPE_IPV4 = 0x800;

#define ET_ANN 0x88B5

#define FUNC_WEIGHTED_SUM_8_TO_8 1
#define FUNC_IDENTITY 2
#define FUNC_RELU 3
#define FUNC_ARGMAX 4
#define FUNC_NORMALIZATION 5
#define FUNC_WEIGHTED_SUM_8_TO_4 6

#define PRECISION 16
#define WORDSIZE 32
#define D_WORDSIZE 64
#define SLACK 8

/*************************************************************************
*********************** H E A D E R S  ***********************************
*************************************************************************/

typedef bit<9>  egressSpec_t;
typedef bit<48> macAddr_t;
typedef bit<32> ip4Addr_t;

header ethernet_t{
    macAddr_t dstAddr;
    macAddr_t srcAddr;
    bit<16>   etherType;
}

header ann_t{
    bit<32> neuron_id;
    bit<WORDSIZE> data_1;
    bit<WORDSIZE> data_2;
    bit<WORDSIZE> data_3;
    bit<WORDSIZE> data_4;
    bit<WORDSIZE> data_5;
    bit<WORDSIZE> data_6;
    bit<WORDSIZE> data_7;
    bit<WOR

### OLD P4 file generation

In [191]:
print("action set_neuron_bias()")
for i in range(8):
  print(f"bit<WORDSIZE> neuron_{i+1}_bias, ")
for i in range(8):
  print(f"meta.neuron_{i+1}_bias = neuron_{i+1}_bias;")


print("action set_n2n_weight()")
for i in range(8):
  for j in range(8):
    print(f"bit<WORDSIZE> n2n_{i+1}_weight_{j+1}, ")
for i in range(8):
  for j in range(8):
    print(f"meta.n2n_{i+1}_weight_{j+1} = n2n_{i+1}_weight_{j+1};")


print("action set_norm_mean_std()")
for i in range(8):
  print(f"bit<WORDSIZE> neuron_{i+1}_mean, ")
for i in range(8):
  print(f"bit<WORDSIZE> neuron_{i+1}_std, ")
for i in range(8):
  print(f"meta.neuron_{i+1}_mean = neuron_{i+1}_mean;")
for i in range(8):
  print(f"meta.neuron_{i+1}_std = neuron_{i+1}_std;")



action set_neuron_bias()
bit<WORDSIZE> neuron_1_bias, 
bit<WORDSIZE> neuron_2_bias, 
bit<WORDSIZE> neuron_3_bias, 
bit<WORDSIZE> neuron_4_bias, 
bit<WORDSIZE> neuron_5_bias, 
bit<WORDSIZE> neuron_6_bias, 
bit<WORDSIZE> neuron_7_bias, 
bit<WORDSIZE> neuron_8_bias, 
meta.neuron_1_bias = neuron_1_bias;
meta.neuron_2_bias = neuron_2_bias;
meta.neuron_3_bias = neuron_3_bias;
meta.neuron_4_bias = neuron_4_bias;
meta.neuron_5_bias = neuron_5_bias;
meta.neuron_6_bias = neuron_6_bias;
meta.neuron_7_bias = neuron_7_bias;
meta.neuron_8_bias = neuron_8_bias;
action set_n2n_weight()
bit<WORDSIZE> n2n_1_weight_1, 
bit<WORDSIZE> n2n_1_weight_2, 
bit<WORDSIZE> n2n_1_weight_3, 
bit<WORDSIZE> n2n_1_weight_4, 
bit<WORDSIZE> n2n_1_weight_5, 
bit<WORDSIZE> n2n_1_weight_6, 
bit<WORDSIZE> n2n_1_weight_7, 
bit<WORDSIZE> n2n_1_weight_8, 
bit<WORDSIZE> n2n_2_weight_1, 
bit<WORDSIZE> n2n_2_weight_2, 
bit<WORDSIZE> n2n_2_weight_3, 
bit<WORDSIZE> n2n_2_weight_4, 
bit<WORDSIZE> n2n_2_weight_5, 
bit<WORDSIZE> n2n_2_

In [192]:
# Normalization
# not even half automated, manual fill method
for i in range(8):
  print(f"                bit<WORDSIZE> operand_a{i+1} = hdr.ann.data_{i+1} << PRECISION;")
for i in range(8):
  print(f"                bit<WORDSIZE> operand_b{i+1} = meta.neuron_{i+1}_mean;")
for i in range(8):
  print(f"                bit<WORDSIZE> sum_result_{i+1} = operand_a{i+1} + operand_b{i+1};")
for i in range(8):
  print(f"                bit<D_WORDSIZE> sum_result_{i+1}_dw = (bit<D_WORDSIZE>) sum_result_{i+1};")
for i in range(8):
  print(f"                if((sum_result_{i+1}_dw & (1 << (WORDSIZE-1))) > 0){{                            // negative number")
  print(f"                    sum_result_{i+1}_dw = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) sum_result_{i+1}_dw;")
  print(f"                }}")

for i in range(8):
  print(f"                bit<D_WORDSIZE> operand_c{i+1} = (bit<D_WORDSIZE>) meta.neuron_{i+1}_std;")
for i in range(8):
  print(f"                if((operand_c{i+1} & (1 << (WORDSIZE-1))) > 0){{                                // negative number")
  print(f"                    operand_c{i+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_c{i+1};")
  print(f"                }}")
for i in range(8):
  print(f"                bit<D_WORDSIZE> norm_result_{i+1} = ((sum_result_{i+1}_dw * operand_c{i+1}) >> PRECISION);")
print(f"                // Resize the data to wordsize to be fowarded")
for i in range(8):
  print(f"                meta.neuron_{i+1}_data = (bit<WORDSIZE>) norm_result_{i+1};")
for i in range(8):
  print(f"                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);")



                bit<WORDSIZE> operand_a1 = hdr.ann.data_1 << PRECISION;
                bit<WORDSIZE> operand_a2 = hdr.ann.data_2 << PRECISION;
                bit<WORDSIZE> operand_a3 = hdr.ann.data_3 << PRECISION;
                bit<WORDSIZE> operand_a4 = hdr.ann.data_4 << PRECISION;
                bit<WORDSIZE> operand_a5 = hdr.ann.data_5 << PRECISION;
                bit<WORDSIZE> operand_a6 = hdr.ann.data_6 << PRECISION;
                bit<WORDSIZE> operand_a7 = hdr.ann.data_7 << PRECISION;
                bit<WORDSIZE> operand_a8 = hdr.ann.data_8 << PRECISION;
                bit<WORDSIZE> operand_b1 = meta.neuron_1_mean;
                bit<WORDSIZE> operand_b2 = meta.neuron_2_mean;
                bit<WORDSIZE> operand_b3 = meta.neuron_3_mean;
                bit<WORDSIZE> operand_b4 = meta.neuron_4_mean;
                bit<WORDSIZE> operand_b5 = meta.neuron_5_mean;
                bit<WORDSIZE> operand_b6 = meta.neuron_6_mean;
                bit<WORDSIZE> operand_b7 = met

In [193]:
# Weighted sum
print(f"                // Aggregation Function = weighted sum = bias + Summation_i=1_to_n(data_i * weight_i)")
print(f"                if(meta.n_received_stimuli == 1){{ // Check if this is the first stimulus in an ANN run")
print(f"                    // If yes, initialize neuron_data with the neuron bias, the neuron bias is added to the accumulator (neuron_data) just once")
for i in range(8):
  print(f"                    meta.neuron_{i+1}_data = meta.neuron_{i+1}_bias;")

print(f"                }}")
print(f"                else{{")
print(f"                    // If not, read the neuron_data value stored in the register")

for i in range(8):
  print(f"                    reg_neuron_{i+1}_data.read(meta.neuron_{i+1}_data, 0);")

print(f"                }}")
print(f"                tab_n2n_weight.apply();	// Get the neuron weights")
print(f"                // Load data and perform sign extension")

for i in range(8):
  print(f"                bit<D_WORDSIZE> operand_b{i+1} = (bit<D_WORDSIZE>) hdr.ann.data_{i+1};")

print(f"                // Sign extension")

for i in range(8):
  print(f"                if((operand_b{i+1} & (1 << (WORDSIZE-1))) > 0){{ // negative number")
  print(f"                    operand_b{i+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_b{i+1};")
  print(f"                }}")

# NxN neurons
for i in range(8):
  print(f"                // Neuron {i+1}   ")
  print(f"                // Pass values to aux variable to be able to operate them")
  for j in range(8):
    print(f"                bit<D_WORDSIZE> operand_a_{i+1}_{j+1} = (bit<D_WORDSIZE>) meta.n2n_{i+1}_weight_{j+1}; ")

  print(f"                // Sign extension")
  for j in range(8):
    print(f"                if((operand_a_{i+1}_{j+1} & (1 << (WORDSIZE-1))) > 0){{ // negative number")
    print(f"                    operand_a_{i+1}_{j+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_a_{i+1}_{j+1};")
    print(f"                }}")

  print(f"                // perform multiplication and shift to transform data back to wordsize")
  for j in range(8):
    print(f"                bit<D_WORDSIZE> res_{i+1}_{j+1} = ((operand_a_{i+1}_{j+1} * operand_b{j+1}) >> PRECISION);")

  # THIS IS NOT FULLY AUTOMATED!
  print(f"                // Compute the summation")
  print(f"                meta.neuron_{i+1}_data = meta.neuron_{i+1}_data + (bit<WORDSIZE>) res_{i+1}_1 + (bit<WORDSIZE>) res_{i+1}_2 + (bit<WORDSIZE>) res_{i+1}_3 + (bit<WORDSIZE>) res_{i+1}_4 + (bit<WORDSIZE>) res_{i+1}_5 + (bit<WORDSIZE>) res_{i+1}_6 + (bit<WORDSIZE>) res_{i+1}_7 + (bit<WORDSIZE>) res_{i+1}_8;")
  print(f"                // Store data to be fowarded")
  print(f"                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);")

                // Aggregation Function = weighted sum = bias + Summation_i=1_to_n(data_i * weight_i)
                if(meta.n_received_stimuli == 1){ // Check if this is the first stimulus in an ANN run
                    // If yes, initialize neuron_data with the neuron bias, the neuron bias is added to the accumulator (neuron_data) just once
                    meta.neuron_1_data = meta.neuron_1_bias;
                    meta.neuron_2_data = meta.neuron_2_bias;
                    meta.neuron_3_data = meta.neuron_3_bias;
                    meta.neuron_4_data = meta.neuron_4_bias;
                    meta.neuron_5_data = meta.neuron_5_bias;
                    meta.neuron_6_data = meta.neuron_6_bias;
                    meta.neuron_7_data = meta.neuron_7_bias;
                    meta.neuron_8_data = meta.neuron_8_bias;
                }
                else{
                    // If not, read the neuron_data value stored in the register
                    reg_neuron_1_data.read

In [194]:
print(f"            else if(meta.agg_func == FUNC_IDENTITY){{")
for i in range(8):
  print(f"                meta.neuron_{i+1}_data = hdr.ann.data_{i+1};")
for i in range(8):
  print(f"                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);")
print(f"            }}")


            else if(meta.agg_func == FUNC_IDENTITY){
                meta.neuron_1_data = hdr.ann.data_1;
                meta.neuron_2_data = hdr.ann.data_2;
                meta.neuron_3_data = hdr.ann.data_3;
                meta.neuron_4_data = hdr.ann.data_4;
                meta.neuron_5_data = hdr.ann.data_5;
                meta.neuron_6_data = hdr.ann.data_6;
                meta.neuron_7_data = hdr.ann.data_7;
                meta.neuron_8_data = hdr.ann.data_8;
                reg_neuron_1_data.write(0, meta.neuron_1_data);
                reg_neuron_2_data.write(0, meta.neuron_2_data);
                reg_neuron_3_data.write(0, meta.neuron_3_data);
                reg_neuron_4_data.write(0, meta.neuron_4_data);
                reg_neuron_5_data.write(0, meta.neuron_5_data);
                reg_neuron_6_data.write(0, meta.neuron_6_data);
                reg_neuron_7_data.write(0, meta.neuron_7_data);
                reg_neuron_8_data.write(0, meta.neuron_8_data);
           

In [195]:
print(f"            else if(meta.agg_func == FUNC_ARGMAX){{")
print(f"                // the data to be fowarded (neuron_1_data) is the ID of the switch with highest value.")
print(f"                // neuron_2_data is the index of the neuron with highest value inside the same switch.")
print(f"                // the highest data (neuron_max_value) is kept to be compared by other neurons.")
print(f"                bit<WORDSIZE> op_a = 0;")
print(f"                bit<WORDSIZE> op_b = 0;")
print(f"                bit<1> op_a_sig = 0;")
print(f"                bit<1> op_b_sig = 0;")
print(f"                if(meta.n_received_stimuli == 1){{")
print(f"                    // if first stimuli, then assume first data received is the higher, then check the remmaining data against it")
print(f"                    // Neuron 1  ")
print(f"                    meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;")
print(f"                    meta.neuron_2_data = 0; // neuron_2_data stores the index of the neuron with highest value within same switch")
print(f"                    meta.neuron_max_value = hdr.ann.data_1;")

for i in range(7):
  print(f"                    // Neuron {i+2}")
  print(f"                    // Check if data is higher than stored max data")
  print(f"                    op_a = hdr.ann.data_{i+2}; 			// op_a is the data being evaluated if it's higher then the stored one (op_b)")
  print(f"                    op_b = meta.neuron_max_value;		// op_b is the store of max value until now")
  print(f"                    op_a_sig = (bit<1>)(op_a & (1 << (WORDSIZE-1)) > 0);")
  print(f"                    op_b_sig = (bit<1>)(op_b & (1 << (WORDSIZE-1)) > 0);")
  print(f"                    // There are two situation in which op_a is bigger then op_b")
  print(f"                    if((op_a_sig == 0) && (op_b_sig  == 1)){{ // The first: if the op_a is positive and op_b is negative")
  print(f"                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;")
  print(f"                        meta.neuron_2_data = {i+1};")
  print(f"                        meta.neuron_max_value = hdr.ann.data_{i+2};")
  print(f"                    }} else if(op_a_sig == op_b_sig && op_a > op_b){{ // The second: if the signal is the same, and op_a > op_b")
  print(f"                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;")
  print(f"                        meta.neuron_2_data = {i+1};")
  print(f"                        meta.neuron_max_value = hdr.ann.data_{i+2};")
  print(f"                    }}")


print(f"                }}")
print(f"                else{{")
print(f"                    // this is wrong!?!? reg_neuron_1_data.read(meta.neuron_1_data, 0);")
print(f"                    reg_neuron_max_value.read(meta.neuron_max_value, 0);")

for i in range(8):
  print(f"                    // Neuron {i+1}")
  print(f"                    // Check if data is higher than stored max data")
  print(f"                    op_a = hdr.ann.data_{i+1}; 			// op_a is the data being evaluated if it's higher then the stored one (op_b)")
  print(f"                    op_b = meta.neuron_max_value;		// op_b is the store of max value until now")
  print(f"                    op_a_sig = (bit<1>)(op_a & (1 << (WORDSIZE-1)) > 0);")
  print(f"                    op_b_sig = (bit<1>)(op_b & (1 << (WORDSIZE-1)) > 0);")
  print(f"                    // There are two situation in which op_a is bigger then op_b")
  print(f"                    if((op_a_sig == 0) && (op_b_sig  == 1)){{ // The first: if the op_a is positive and op_b is negative")
  print(f"                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;")
  print(f"                        meta.neuron_2_data = {i};")
  print(f"                        meta.neuron_max_value = hdr.ann.data_{i+1};")
  print(f"                    }} else if(op_a_sig == op_b_sig && op_a > op_b){{ // The second: if the signal is the same, and op_a > op_b")
  print(f"                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;")
  print(f"                        meta.neuron_2_data = {i};")
  print(f"                        meta.neuron_max_value = hdr.ann.data_{i+1};")
  print(f"                    }}")


print(f"                }}")
print(f"                reg_neuron_1_data.write(0, meta.neuron_1_data);")
print(f"                reg_neuron_2_data.write(0, meta.neuron_2_data);")
print(f"                reg_neuron_max_value.write(0, meta.neuron_max_value);")
print(f"                // in the argmax function, neuron 3 data and neuron 4 data are not needed, set to 99 just for testing. Could use a different header for this layer so we don't have empty fileds.")

for i in range(6):
  print(f"                meta.neuron_{i+3}_data = 99;")
for i in range(6):
  print(f"                reg_neuron_{i+3}_data.write(0, meta.neuron_{i+3}_data);")
print(f"            }}")

            else if(meta.agg_func == FUNC_ARGMAX){
                // the data to be fowarded (neuron_1_data) is the ID of the switch with highest value.
                // neuron_2_data is the index of the neuron with highest value inside the same switch.
                // the highest data (neuron_max_value) is kept to be compared by other neurons.
                bit<WORDSIZE> op_a = 0;
                bit<WORDSIZE> op_b = 0;
                bit<1> op_a_sig = 0;
                bit<1> op_b_sig = 0;
                if(meta.n_received_stimuli == 1){
                    // if first stimuli, then assume first data received is the higher, then check the remmaining data against it
                    // Neuron 1  
                    meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                    meta.neuron_2_data = 0; // neuron_2_data stores the index of the neuron with highest value within same switch
                    meta.neuron_max_value = hdr.ann.data_1;
             

In [196]:
#activation funcitons relu and identity
print(f"                if(meta.activation_func == FUNC_RELU){{")
for i in range(8):
  print(f"                    if(meta.neuron_{i+1}_data & (1 << (WORDSIZE-1)) > 0){{     // Relu: if negative, set data to 0")
  print(f"                        meta.neuron_{i+1}_data = 0;")
  print(f"                    }}")

for i in range(8):
  print(f"                    hdr.ann.data_{i+1} = meta.neuron_{i+1}_data;                    // Overwrite the fields in the ANN header")

print(f"                }}")
print(f"                else if(meta.activation_func == FUNC_IDENTITY){{")
for i in range(8):
  print(f"                    hdr.ann.data_{i+1} = meta.neuron_{i+1}_data;                    // Overwrite the fields in the ANN header")

print(f"                }}")

                if(meta.activation_func == FUNC_RELU){
                    if(meta.neuron_1_data & (1 << (WORDSIZE-1)) > 0){     // Relu: if negative, set data to 0
                        meta.neuron_1_data = 0;
                    }
                    if(meta.neuron_2_data & (1 << (WORDSIZE-1)) > 0){     // Relu: if negative, set data to 0
                        meta.neuron_2_data = 0;
                    }
                    if(meta.neuron_3_data & (1 << (WORDSIZE-1)) > 0){     // Relu: if negative, set data to 0
                        meta.neuron_3_data = 0;
                    }
                    if(meta.neuron_4_data & (1 << (WORDSIZE-1)) > 0){     // Relu: if negative, set data to 0
                        meta.neuron_4_data = 0;
                    }
                    if(meta.neuron_5_data & (1 << (WORDSIZE-1)) > 0){     // Relu: if negative, set data to 0
                        meta.neuron_5_data = 0;
                    }
                    if(meta.neuron_6_data & (

### Topology file generator for topology.json


In [197]:
# #definition of topology
# topology = {
#     'hosts': {
#         'h1': {'ip': '10.0.1.1/24', 'mac': '08:00:00:00:01:11',
#                 'commands':['route add default gw 10.0.1.10 dev eth0',
#                             'arp -i eth0 -s 10.0.1.10 08:00:00:00:01:00']},
#         'h2': {'ip': '10.0.2.2/24', 'mac': '08:00:00:00:02:22',
#                 'commands':['route add default gw 10.0.2.20 dev eth0',
#                             'arp -i eth0 -s 10.0.2.20 08:00:00:00:02:00']}
#     },
#     'switches': {},
#     'links': []
# }

# #populate switches dict
# for i in range(number_of_attributes):
#   switch_name = 's%d'%(i+1)
#   switch_content = { 'runtime_json' : 'topology/s%d'%(i+1) +'-runtime.json' }
#   topology['switches'][switch_name] = switch_content

# for i in range(hidden_layer_nodes):
#   switch_name = 's%d'%(i+51)
#   switch_content = { 'runtime_json' : 'topology/s%d'%(i+51) +'-runtime.json' }
#   topology['switches'][switch_name] = switch_content

# for i in range(output_layer_nodes):
#   switch_name = 's%d'%(i+101)
#   switch_content = { 'runtime_json' : 'topology/s%d'%(i+101) +'-runtime.json' }
#   topology['switches'][switch_name] = switch_content

# topology['switches']['s126'] = { 'runtime_json' : 'topology/s126-runtime.json' }

# #populate links list
# for i in range(number_of_attributes):
#   topology['links'].append(["h1", "s%d"%(i+1)+"-p1"])

# for i in range(number_of_attributes):
#   for j in range(hidden_layer_nodes):
#     topology['links'].append(["s%d"%(i+1)+"-p%d"%(j+51), "s%d"%(j+51)+"-p%d"%(i+1)])

# for i in range(hidden_layer_nodes):
#   for j in range(output_layer_nodes):
#     topology['links'].append(["s%d"%(i+51)+"-p%d"%(j+101), "s%d"%(j+101)+"-p%d"%(i+51)])

# for i in range(output_layer_nodes):
#   topology['links'].append(["s%d"%(i+101)+"-p126", "s126-p%d"%(i+101)])

# topology['links'].append(["s126-p2", "h2"])

# display(topology)

# topology_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/topology.json"
# with open(topology_title, "w") as f:
#   json.dump(topology, f, indent = 2)


### Jinja template to generate ann.p4, ann-header.py and ann_tester.py

In [198]:
# environment = Environment(loader=FileSystemLoader("/content/drive/MyDrive/network-traffic-classification-main/jinja/templates/"))
# template = environment.get_template("ANN_tester_b.py")
# content = template.render(
#     wordsize= wordsize,
#     precision= precision,
#     integer= wordsize - precision,
#     fraction = precision,
#     slack = slack,
#     df_number = df_number,
#     number_of_attributes = number_of_attributes

# )
# filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN_tester_b.py"
# with open(filename, mode="w", encoding="utf-8") as message:
#     message.write(content)
#     print(f"... wrote {filename}")

# ###################################################
# template = environment.get_template("ANN_hdrs.py")
# content = template.render(
#     wordsize= wordsize,
#     slack = slack,
# )
# filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN_hdrs.py"
# with open(filename, mode="w", encoding="utf-8") as message:
#     message.write(content)
#     print(f"... wrote {filename}")

# ###################################################
# template = environment.get_template("ANN.p4")
# content = template.render(
#     wordsize= wordsize,
#     precision= precision,
#     double = wordsize*2,
#     slack = slack,
# )
# filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN.p4"
# with open(filename, mode="w", encoding="utf-8") as message:
#     message.write(content)
#     print(f"... wrote {filename}")

In [199]:
biases = norm_layer_biases
for i in range(number_of_attributes):
  inverted_sqrt_bias = (math.sqrt(biases[i])) ** -1
  print(f"bias: {biases[i]}, inverted_sqrt_bias:{inverted_sqrt_bias}")

bias: 193688.4375, inverted_sqrt_bias:0.0022722083536081167
bias: 55130.38671875, inverted_sqrt_bias:0.004258969015574933
bias: 166393.203125, inverted_sqrt_bias:0.0024515017575000276
bias: 453339520.0, inverted_sqrt_bias:4.696650135517624e-05
bias: 224058.0625, inverted_sqrt_bias:0.0021126118520492813
bias: 224139.796875, inverted_sqrt_bias:0.0021122266264974768
bias: 64357.88671875, inverted_sqrt_bias:0.003941841091834236
bias: 633155520.0, inverted_sqrt_bias:3.974154998053779e-05


In [200]:
# #expected stimuli
# #hidden
# sum([1<<(i + 1) for i in range(n)])

# #output
# sum([1<<(i + 51) for i in range(n)])




# #expected stimuli
# import sys

# def main(neurons):
#     neurons = [int(x) for x in neurons.split(",")]
#     expected_stimuli = 0
#     for x in neurons:
#         expected_stimuli = expected_stimuli | 1 << x

#     print("dec:", expected_stimuli)
#     print("bin:", bin(expected_stimuli))

# if __name__ == '__main__':
#     main(sys.argv[1])

# #6
# #15762598695796736